In [1]:
import numpy as np  
import sys,os  
caffe_root = '/data/ssd-caffe/py2_yolov3_caffe/'
sys.path.insert(0, caffe_root + 'python')  
import caffe

#train_proto = 'yolov3-tiny-ncs-without-last-maxpool.prototxt'
#train_model = 'snapshot/mobilenet_iter_7000.caffemodel'  #should be your snapshot caffemodel
train_proto = 'yolov3.prototxt'
deploy_proto = 'yolov3_merged_bn.prototxt'
'''if(len(sys.argv) < 2):
    print("please input caffemodel file from commandLine")
    exit(1)
train_model = sys.argv[1]'''

#deploy_proto = 'yolov3-tiny-ncs-without-last-maxpool-merg-batchnorm.prototxt'  
#save_model = 'iter_7000_MobileNetSSD_deploy.caffemodel'
#save_model = train_model.split('/')[-1] + "MobileNetSSD_deploy.caffemodel"
save_model = "generated_merge_yolov3_header_int.caffemodel"
save_weights = "header_int_aaa_yolov3.weights"
#global buf #----why can not work ?

def merge_bn(net, nob):
    '''
    merge the batchnorm, scale layer weights to the conv layer, to  improve the performance
    var = var + scaleFacotr
    rstd = 1. / sqrt(var + eps)
    w = w * rstd * scale
    b = (b - mean) * rstd * scale + shift
    '''
    buf = [0, 2, 0, 960000, 0]
    for key in net.params.iterkeys():
        print("key:" + str(key))
        if type(net.params[key]) is caffe._caffe.BlobVec:
            print("is caffe._caffe.BlobVec")
            if key.endswith("-bn") or key.endswith("-scale"):
                continue
            else:
                conv = net.params[key]
                print(conv[0].data)
                print("merge layer {0}".format(conv))
                if not net.params.has_key(key[:-5] + "-bn"):
                    print("not need bn, copy w and b")
                    for i, w in enumerate(conv):
                        #print("i={0}".format(i))
                        nob.params[key][i].data[...] = w.data
                        #print(w.data.shape)
                    temp_array = (conv[1].data).reshape(-1)                       
                    buf = buf + temp_array.tolist()
                    print(len(buf))
                    temp_array = (conv[0].data).reshape(-1)                       
                    buf = buf + temp_array.tolist()
                    print(len(buf))
                else:
                    
                    print("need bn, really start bn and scale")
                    bn = net.params[key[:-5] + "-bn"]
                    scale = net.params[key[:-5] + "-scale"]
                    wt = conv[0].data
                    channels = wt.shape[0]
                    bias = np.zeros(wt.shape[0])
                    if len(conv) > 1:
                        bias = conv[1].data
                    mean = bn[0].data
                    var = bn[1].data
                    scalef = bn[2].data
                    print('scalef = {0}'.format(scalef))

                    scales = scale[0].data
                    shift = scale[1].data

                    if scalef != 0:
                        scalef = 1. / scalef
                    mean = mean * scalef
                    var = var * scalef
                    rstd = 1. / np.sqrt(var + 1e-5)
                    rstd1 = rstd.reshape((channels,1,1,1))
                    scales1 = scales.reshape((channels,1,1,1))
                    wt = wt * rstd1 * scales1
                    bias = (bias - mean) * rstd * scales + shift                   
                    nob.params[key][0].data[...] = wt                   
                    nob.params[key][1].data[...] = bias
                    
                    print(bias.shape)
                    temp_array = bias.reshape(-1)
                    buf = buf + temp_array.tolist()
                    print(len(buf))
                    
                    print(wt.shape)
                    temp_array = wt.reshape(-1)
                    buf = buf + temp_array.tolist()
                    print(len(buf))
    
    weights_header = np.array(buf[:5], dtype=np.int32)
    weights_merged_bn = np.array(buf[5:], dtype=np.float32)
    print(weights_header)
    print(weights_merged_bn)
    fp = open(save_weights, "wb")
    weights_header.tofile(fp)
    weights_merged_bn.tofile(fp)
    #np.save('npsaved_file', weights_merged_bn)

#net = caffe.Net(train_proto, train_model, caffe.TRAIN)
caffe.set_mode_cpu()
#net = caffe.Net(train_proto, 'Jenerated_nolastpooling.caffemodel', caffe.TEST)
net = caffe.Net(train_proto, 'yolov3.caffemodel', caffe.TEST)
net_deploy = caffe.Net(deploy_proto, caffe.TEST)  
print('benchmark')
merge_bn(net, net_deploy)
#net_deploy.save(save_model)


benchmark
key:layer1-conv
is caffe._caffe.BlobVec
[[[[ -7.39838630e-02   1.79278374e-01  -3.00073363e-02]
   [  9.11507085e-02  -1.68761816e-02   9.80103165e-02]
   [  2.71728705e-03  -7.27469146e-01   2.75320895e-02]]

  [[ -4.35802005e-02  -1.08121470e-01  -2.76579820e-02]
   [  3.74525450e-02  -1.49165183e-01   6.87774867e-02]
   [  4.02355902e-02  -4.11971033e-01   6.21622801e-02]]

  [[  2.63345968e-02   1.10361390e-01  -7.16836452e-02]
   [  6.07688501e-02   5.06702401e-02   3.66386212e-02]
   [ -7.10724574e-03  -4.44822967e-01   1.44725703e-02]]]


 [[[ -1.65614009e-01  -1.39581084e-01  -7.55509213e-02]
   [ -2.65302099e-02  -2.58116961e-01  -2.65929028e-02]
   [ -2.87525505e-02   3.78144085e-02  -5.21513298e-02]]

  [[ -1.81117013e-01  -1.55092314e-01   1.05160356e-01]
   [ -3.05253938e-02  -2.60974079e-01   6.87906742e-02]
   [  4.11110148e-02   8.76532122e-02   5.64741110e-03]]

  [[ -2.10971773e-01  -1.75061151e-01   3.37304771e-02]
   [ -9.25346464e-02  -2.42266491e-01   1.

1262245
key:layer20-bn
is caffe._caffe.BlobVec
key:layer20-scale
is caffe._caffe.BlobVec
key:layer21-conv
is caffe._caffe.BlobVec
[[[[ -1.60650536e-02  -4.22709472e-02  -1.89879984e-02]
   [ -9.67263896e-03  -1.25307851e-02  -2.04524398e-02]
   [ -6.84060389e-03  -6.99626887e-03   2.90908618e-03]]

  [[  1.48101747e-02  -2.72493735e-02  -2.05576774e-02]
   [ -1.02674989e-02  -3.12475208e-03  -1.92529596e-02]
   [  3.13309859e-03   3.39980163e-02   2.08572857e-02]]

  [[ -3.46488133e-02   7.18490127e-03  -3.61631401e-02]
   [ -1.69454589e-02   1.52634859e-01  -1.89698711e-02]
   [  3.84178432e-03   3.86432521e-02  -8.12466256e-03]]

  ..., 
  [[  1.07416315e-02  -2.73604617e-02  -1.85779831e-03]
   [  4.02172701e-03  -6.68995902e-02   1.90265700e-02]
   [ -2.62022056e-02   2.73828767e-03   1.25747826e-03]]

  [[  3.14529650e-02  -1.05330246e-02   4.19547185e-02]
   [ -1.14993611e-02  -1.57325324e-02   2.15477198e-02]
   [ -1.29780145e-02  -1.02680009e-02   2.16184370e-02]]

  [[ -1.8406

1918373
key:layer26-bn
is caffe._caffe.BlobVec
key:layer26-scale
is caffe._caffe.BlobVec
key:layer27-conv
is caffe._caffe.BlobVec
[[[[  1.53335524e-04   3.68604288e-02  -1.61810480e-02]
   [  5.24826124e-02   9.84072834e-02   9.76070948e-03]
   [ -1.25282649e-02   5.22568300e-02  -4.99588065e-03]]

  [[ -3.80915403e-02  -1.38448533e-02   3.98042379e-03]
   [ -4.61023487e-02  -2.30297316e-02   1.96026769e-02]
   [ -4.25491482e-02  -1.18948789e-02  -4.72433283e-04]]

  [[ -1.26187112e-02  -6.21276442e-03  -2.56477166e-02]
   [  1.05350539e-02   4.35168818e-02  -4.47546430e-02]
   [  1.10604279e-02  -2.56904960e-02  -1.53388223e-02]]

  ..., 
  [[ -9.98996943e-03   1.50496140e-02   5.60350716e-03]
   [  1.00567238e-02   4.39856127e-02   3.35869268e-02]
   [ -1.94997806e-03   3.63131636e-03   6.46912120e-03]]

  [[  2.42243521e-02  -1.23271113e-02   1.60378702e-02]
   [  3.45844142e-02  -3.27273458e-02   2.42533274e-02]
   [  3.37389372e-02  -3.83745767e-02   7.97875784e-03]]

  [[ -3.0254

2574501
key:layer32-bn
is caffe._caffe.BlobVec
key:layer32-scale
is caffe._caffe.BlobVec
key:layer33-conv
is caffe._caffe.BlobVec
[[[[  6.83887023e-03  -3.59983323e-03   1.40804490e-02]
   [  2.65428331e-02  -4.40672925e-03  -3.53686803e-04]
   [  2.07286980e-03   1.93759182e-03  -2.86722207e-03]]

  [[ -3.66617693e-03   1.07971318e-02  -1.61622129e-02]
   [  8.63033999e-03  -3.33902077e-03  -2.36835219e-02]
   [ -2.00963467e-02  -1.59439947e-02   1.38810882e-02]]

  [[ -3.92736262e-03  -2.55529620e-02   1.36308027e-02]
   [  1.80441011e-02  -1.25550432e-02   1.74642168e-02]
   [ -3.60290036e-02  -6.86451942e-02  -1.01535767e-02]]

  ..., 
  [[  2.43523121e-02   2.53526829e-02   3.55695444e-03]
   [  4.46243957e-02   8.22705701e-02   1.87219810e-02]
   [ -3.08227036e-02   4.73958515e-02  -4.11474518e-02]]

  [[  3.37428078e-02  -6.77969083e-02   2.62748413e-02]
   [ -2.59290151e-02  -3.69419679e-02  -1.54597883e-03]
   [ -1.11433715e-02  -1.77845508e-02  -7.40783149e-03]]

  [[  2.4529

3197733
key:layer36-bn
is caffe._caffe.BlobVec
key:layer36-scale
is caffe._caffe.BlobVec
key:layer38-conv
is caffe._caffe.BlobVec
[[[[ 0.03046795  0.02055201  0.00584922]
   [ 0.01058268 -0.02074889  0.00854579]
   [ 0.02366639 -0.00046508  0.01558431]]

  [[ 0.0056556   0.00606483  0.02041344]
   [ 0.01154157  0.01277078  0.02311315]
   [-0.00353468 -0.00439765  0.0114432 ]]

  [[-0.00049473 -0.03149421 -0.00803938]
   [-0.00025003 -0.04039459 -0.04329225]
   [-0.00553915 -0.04195739 -0.04308344]]

  ..., 
  [[-0.01882067 -0.00302955  0.00629469]
   [-0.01256708 -0.0055719  -0.00915665]
   [ 0.00196637  0.00192617 -0.01838697]]

  [[-0.03097085 -0.02851766 -0.01604991]
   [-0.01512747 -0.06474902 -0.03109118]
   [ 0.00137631 -0.05069508 -0.05071376]]

  [[ 0.02030139 -0.00916233 -0.019511  ]
   [ 0.00842957 -0.02980906 -0.02274878]
   [ 0.01710014 -0.02061922 -0.00985587]]]


 [[[-0.00752268  0.00014494  0.0065721 ]
   [ 0.01740051 -0.00551826  0.0092672 ]
   [-0.0420515  -0.00472925 

...

58070052
key:layer89-bn
is caffe._caffe.BlobVec
key:layer89-scale
is caffe._caffe.BlobVec
key:layer90-conv
is caffe._caffe.BlobVec
[[[[-0.04561019]]

  [[ 0.04164124]]

  [[-0.00337723]]

  ..., 
  [[ 0.03151703]]

  [[-0.00522386]]

  [[ 0.05947917]]]


 [[[ 0.00723647]]

  [[ 0.01003107]]

  [[-0.01306361]]

  ..., 
  [[ 0.09413268]]

  [[-0.01773269]]

  [[-0.01789343]]]


 [[[ 0.05286223]]

  [[-0.0507841 ]]

  [[ 0.01170997]]

  ..., 
  [[-0.01191004]]

  [[-0.00793742]]

  [[-0.00459767]]]


 ..., 
 [[[-0.03476092]]

  [[ 0.0524152 ]]

  [[ 0.09960615]]

  ..., 
  [[ 0.06830613]]

  [[-0.0088802 ]]

  [[-0.04450806]]]


 [[[ 0.00028646]]

  [[-0.00442939]]

  [[ 0.07442693]]

  ..., 
  [[-0.00697295]]

  [[-0.01308924]]

  [[ 0.00089222]]]


 [[[ 0.04866096]]

  [[-0.02333969]]

  [[-0.0672932 ]]

  ..., 
  [[ 0.06622211]]

  [[-0.00504035]]

  [[-0.07230638]]]]
merge layer <caffe._caffe.BlobVec object at 0x7f5ffba7f2f0>
need bn, really start bn and scale
scalef = [ 1.]
(256,)
58

(256, 512, 1, 1)
59512868
key:layer92-bn
is caffe._caffe.BlobVec
key:layer92-scale
is caffe._caffe.BlobVec
key:layer93-conv
is caffe._caffe.BlobVec
[[[[ -1.64486433e-03  -5.29840123e-03   2.01598322e-03]
   [ -3.92492348e-03  -1.48713477e-02   3.59049812e-03]
   [ -2.09431350e-02  -1.58720016e-02  -1.40438396e-02]]

  [[ -2.09579198e-03  -2.34872010e-03   8.88291188e-03]
   [ -4.46352223e-03   1.96460728e-03  -5.02857240e-03]
   [  9.32933390e-03   4.83239908e-03  -2.69677956e-03]]

  [[  2.22143456e-02   1.53990146e-02   2.40483265e-02]
   [  1.86657310e-02   3.68912369e-02   3.32688205e-02]
   [  7.08921952e-03   2.36008149e-02   1.10176923e-02]]

  ..., 
  [[ -1.17108254e-02  -4.24674340e-03  -3.60955261e-02]
   [ -2.75853947e-02  -1.96297653e-02  -2.82502379e-02]
   [ -5.15655428e-03  -2.27042101e-03  -1.21257445e-02]]

  [[ -1.01463450e-02  -3.60792561e-04  -1.11303793e-03]
   [ -9.42728075e-05   5.30013640e-04  -2.73323478e-03]
   [ -1.96830463e-03  -5.40247001e-03  -1.63157545e-

key:layer97-bn
is caffe._caffe.BlobVec
key:layer97-scale
is caffe._caffe.BlobVec
key:layer100-conv
is caffe._caffe.BlobVec
[[[[-0.09996903]]

  [[-0.0818598 ]]

  [[ 0.01806047]]

  ..., 
  [[-0.01153516]]

  [[-0.0652115 ]]

  [[-0.04853309]]]


 [[[-0.09351571]]

  [[-0.0272919 ]]

  [[ 0.05261033]]

  ..., 
  [[-0.06516536]]

  [[ 0.05034191]]

  [[-0.02307437]]]


 [[[-0.03814822]]

  [[ 0.04390454]]

  [[-0.03961139]]

  ..., 
  [[ 0.03467336]]

  [[-0.02628701]]

  [[-0.0042641 ]]]


 ..., 
 [[[ 0.03098625]]

  [[ 0.02969943]]

  [[-0.06200998]]

  ..., 
  [[ 0.01736246]]

  [[-0.03452953]]

  [[-0.01380638]]]


 [[[ 0.03474011]]

  [[-0.0601607 ]]

  [[ 0.05464571]]

  ..., 
  [[ 0.00970156]]

  [[-0.05016552]]

  [[ 0.0078127 ]]]


 [[[ 0.00094878]]

  [[ 0.05892649]]

  [[ 0.0827622 ]]

  ..., 
  [[-0.01773908]]

  [[ 0.00185983]]

  [[ 0.03143118]]]]
merge layer <caffe._caffe.BlobVec object at 0x7f5ffba7f9f0>
need bn, really start bn and scale
scalef = [ 1.]
(128,)
60856867
(

61201315
(128, 256, 1, 1)
61234083
key:layer102-bn
is caffe._caffe.BlobVec
key:layer102-scale
is caffe._caffe.BlobVec
key:layer103-conv
is caffe._caffe.BlobVec
[[[[-0.01968216 -0.01739578 -0.01222171]
   [ 0.02350735  0.05059465  0.0273677 ]
   [ 0.00397436  0.03989173  0.01057773]]

  [[-0.0029919   0.00035294 -0.01114118]
   [ 0.01020336  0.03432763  0.01971231]
   [ 0.03285702  0.05132195  0.01112326]]

  [[ 0.02531511 -0.00621857  0.01947686]
   [ 0.02286083  0.00653401  0.01544098]
   [-0.02127757 -0.05329317 -0.01659427]]

  ..., 
  [[-0.01260181  0.01412363  0.00170186]
   [-0.02590236 -0.04289606 -0.03317509]
   [-0.02229278 -0.0190168  -0.00220265]]

  [[ 0.01654764  0.00622018  0.01944306]
   [ 0.00066555 -0.01991284  0.00902879]
   [-0.02130481 -0.03385578  0.0076829 ]]

  [[-0.02643882 -0.03426461 -0.02928174]
   [-0.00483777  0.01733922 -0.00483448]
   [ 0.02592747  0.02052074  0.01900488]]]


 [[[ 0.03330369  0.05302063  0.04081357]
   [-0.00644874  0.01015299 -0.03739884

61562403
(256, 128, 3, 3)
61857315
key:layer105-bn
is caffe._caffe.BlobVec
key:layer105-scale
is caffe._caffe.BlobVec
key:layer106-conv
is caffe._caffe.BlobVec
[[[[ 0.00548067]]

  [[ 0.0141976 ]]

  [[-0.13984175]]

  ..., 
  [[ 0.03671292]]

  [[ 0.00509541]]

  [[-0.00147751]]]


 [[[ 0.03030118]]

  [[ 0.00511011]]

  [[-0.03207429]]

  ..., 
  [[-0.04042087]]

  [[-0.03459209]]

  [[-0.00776507]]]


 [[[ 0.01569151]]

  [[-0.01077112]]

  [[-0.04119657]]

  ..., 
  [[-0.09545335]]

  [[ 0.00824186]]

  [[-0.00460996]]]


 ..., 
 [[[-0.04378218]]

  [[-0.02020539]]

  [[-0.02092429]]

  ..., 
  [[ 0.01633202]]

  [[-0.05394031]]

  [[-0.04648834]]]


 [[[-0.02608934]]

  [[-0.04344816]]

  [[-0.00497148]]

  ..., 
  [[-0.0442131 ]]

  [[ 0.00637062]]

  [[-0.10562138]]]


 [[[-0.0068372 ]]

  [[-0.0140844 ]]

  [[-0.01951117]]

  ..., 
  [[-0.03373355]]

  [[-0.04969028]]

  [[-0.11880384]]]]
merge layer <caffe._caffe.BlobVec object at 0x7f5ffba81210>
not need bn, copy w and b
6185

In [ ]:
help(np.ndarray.tofile)